In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import socket
import struct

In [2]:
dataset = pd.read_csv('dataset.csv')

In [3]:
dataset.sample(3)

,Unnamed: 0,transaction_id,ip,device_id,device_type,tran_code,mcc,client_id,card_type,pin_inc_count,card_status,expiration_date,datetime,sum,oper_type,balance
422,422,1428598,213.151.0.165,1012.0,atm,10,9663,943042597,DEBIT,0,active,2030-01-04,2022-03-29 23:54:55,759.12,in_acc,557025.74
19817,19817,7340909,195.143.0.90,1369.0,pos trm,-1,6805,420262424,DEBIT,1,active,2030-05-03,2024-01-19 22:46:34,953.56,err,718409.40
5036,5036,9398510,188.0.166.15,1297.0,ATM,451,7752,429842896,CREDIT,0,active,2025-02-01,2024-09-17 12:05:33,363.99,payment,263087.54


#### Более подробнее рассмотрим колонки с типом данных "оbject"


In [4]:
dataset.loc[:, dataset.dtypes == 'object']

,ip,device_type,card_type,card_status,expiration_date,datetime,oper_type
0,91.107.0.90,prtbl trm,DEBIT,active,2030-06-10,2022-03-09 09:56:51,transfer
1,37.0.127.150,port_trm,DEBIT,active,2030-06-10,2022-03-10 09:38:45,transfer
2,185.233.0.240,atm,DEBIT,active,2030-06-10,2022-03-21 23:29:57,diff_cntry
3,91.107.0.90,prtbl trm,DEBIT,active,2030-06-10,2022-03-24 02:18:28,transfer
4,188.43.0.135,ATM,DEBIT,active,2030-06-10,2022-07-07 09:22:18,in_acc
...,...,...,...,...,...,...,...
53074,91.122.0.15,atm,CREDIT,act,2028-01-25,2024-10-28 09:38:58,country_transfer
53075,91.122.0.15,atm,CREDIT,active,2028-08-16,2024-11-13 03:44:07,add_on_acc
53076,2.94.0.240,atm,DEBIT,active,2030-03-08,2024-11-04 14:41:05,in_acc
53077,188.0.162.75,atm,CREDIT,act,2025-06-04,2024-11-23 22:36:43,diff_cntry


In [5]:
print(dataset["device_type"].unique(), dataset["card_type"].unique(), dataset["card_status"].unique()
      ,dataset["oper_type"].unique(), dataset["ip"].dtype, dataset["expiration_date"].dtype, dataset["datetime"].dtype,  sep="\n")


['prtbl trm' 'port_trm' 'atm' 'ATM' 'cash_out' 'Portable term' 'cash_in'
 'pos trm']
['DEBIT' 'CREDIT']
['active' 'act' 'blk' 'blocked']
['transfer' 'diff_cntry' 'in_acc' 'err' 'country_transfer' 'add_on_acc'
 'payment' 'in' 'out' 'bad' 'from_acc' 'err_code' 'decrease_on_acc' 'blk'
 'blocked']
object
object
object


Очень много уникальных значений которые одинаковые по смыслу. Айпишник, две колонки со временем имеют тип "object"

##### Функции преобразования


def device(dev): - device_type

def active(card): - card_status

def ip_to_int(ip): - ip

def trans_oper(oper): - oper_type



In [6]:
def device(dev):
    match dev:
        case 'prtbl trm':
            return "portable_term"
        case 'port_trm':
            return "portable_term"
        case 'Portable term':
            return "portable_term"
        case 'ATM':
            return "atm"
        case 'pos trm':
            return "portable_term"
        case 'cash_out':
            return "unknown"
        case 'cash_in':
            return "unknown"

        case _:
            return dev

def active(card):
    match card:
        case 'act':
            return "active"
        case "blk":
            return "blocked"
        case _:
            return card

def ip_to_int(ip):
    return struct.unpack("!I", socket.inet_aton(ip))[0]


def trans_oper(oper):
    match oper:
        case 'transfer':
            return "transfer"
        case 'diff_cntry':
            return "international_transfer"
        case 'in_acc':
            return "incoming"
        case 'err':
            return "error"
        case 'country_transfer':
            return "international_transfer"
        case 'add_on_acc':
            return "incoming"
        case 'payment':
            return "payment"
        case 'in':
            return "incoming"
        case 'out':
            return "outgoing"
        case 'bad':
            return "bad_transaction"
        case 'from_acc':
            return "outgoing"
        case 'err_code':
            return "error"
        case 'error_code':
            return "error"
        case 'decrease_on_acc':
            return "outgoing"
        case 'blk':
            return "blocked"
        case 'blocked':
            return "blocked"
        case _:
            return oper

In [7]:
dataset["card_type"] = dataset["card_type"].str.lower()
dataset["device_type"] = dataset["device_type"].apply(device)
dataset["card_status"] = dataset["card_status"].apply(active).map({'active': 1, 'blocked': 0}).astype(int)
dataset["ip"] = dataset["ip"].apply(ip_to_int).astype(int)
dataset["expiration_date"] = pd.to_datetime(dataset["expiration_date"]).dt.date.astype('datetime64[ns]')
dataset["datetime"] = pd.to_datetime(dataset["datetime"]).astype('datetime64[ns]')
dataset["oper_type"] = dataset["oper_type"].apply(trans_oper)
print(dataset["device_type"].unique(), dataset["card_type"].unique(), dataset["card_status"].unique()
      , dataset["oper_type"].unique(), dataset["ip"].dtype, dataset["expiration_date"].dtype, dataset["datetime"].dtype,
      sep="\n")


['portable_term' 'atm' 'unknown']
['debit' 'credit']
[1 0]
['transfer' 'international_transfer' 'incoming' 'error' 'payment'
 'outgoing' 'bad_transaction' 'blocked']
int64
datetime64[ns]
datetime64[ns]


#### Преобразованые данные имеют уже структурированный вид.

'cash_out', 'cash_in' в dataset["device_type"], были преобразованны в 'unknown' так как не относится к типу устройства




In [8]:
dataset.loc[:, dataset.dtypes == 'object']

,device_type,card_type,oper_type
0,portable_term,debit,transfer
1,portable_term,debit,transfer
2,atm,debit,international_transfer
3,portable_term,debit,transfer
4,atm,debit,incoming
...,...,...,...
53074,atm,credit,international_transfer
53075,atm,credit,incoming
53076,atm,debit,incoming
53077,atm,credit,international_transfer


In [9]:
dataset.to_csv('normal.csv', index=False)